In [1]:
import sqlite3
conn = sqlite3.connect('main.db') # insert user/password database file name here
cursor = conn.cursor() # create cursor object

In [71]:
def register(username:str, password:str, fName:str, lName:str) -> str:
    cursor.execute(f"SELECT * FROM Users WHERE Username=?;", [username])
    if len(cursor.fetchall()) > 0 :
        return "Username already exists."
    else:
        cursor.execute("INSERT INTO Users VALUES (?, ?, ?, ?);", [username, password, fName, lName])
        cursor.execute("INSERT INTO Scores VALUES (?,?);", [username, 0])
        conn.commit()
        return "Successful"

In [28]:
def login(user, password):
    cursor.execute("SELECT * FROM Users WHERE Username = ?",[user]) # searches the database for the entered username
    result = cursor.fetchall()
    print(result)
    if len(result) == 0: # checks if an account with such a username exists
        return "No Username"
    correct_password = result[0][1] # assumes the correct password is kept in column index 1 of the database 
    if password == correct_password: # compares entered password and possible correct passwords (assuming duplicate usernames are allowed)
        return "Logged In"
    return "Incorrect Password"

In [19]:
def correctness(questionNum, answer): #answer is entered as string or as 2-d array entry
    search_num = questionNum
    cursor.execute("SELECT * FROM Questions WHERE QuestionID = ?",[search_num])
    result = cursor.fetchall()
    if answer == result[0][4]:  # checks if entry equals the correct answer
        return result[0][3] # replace ? with the column index of the point value assigned to the question
    else:
        return 0

In [66]:
import random
def shuffle(question_num):
    cursor.execute("SELECT * FROM Questions WHERE QuestionID = (?)",[question_num]) #parses database for question id
    result = cursor.fetchall()
    question_list = [result[0][4], result[0][5], result[0][6], result[0][7]] #assembles list of question options
    random.shuffle(question_list) #shuffles list
    return question_list #returns shuffled list

In [54]:
def addPoints(questionId, answer, user): #note, as of 10/31, you can answer the same question multiple times and farm infinite points.
    newPoints = correctness(questionId, answer) #finds number of points earned from previous function
    cursor.execute("SELECT * FROM Scores WHERE Username = (?)",[user]) #finds number of points from the points table
    result = cursor.fetchall()
    updated_points = result[0][1] + newPoints #adds old points and newly earned points
    cursor.execute("UPDATE Scores SET Points = (?) WHERE Username = (?)", [updated_points, user]) #updates table value for username
    conn.commit()